In [1]:
## 0/1ナップザック問題
class Item(object):
    def __init__(self, n, v, w):
        self.name = n
        self.weight = w
        self.name = n
        self.value = v
    def getName(self):
        return self.name
    def getValue(self):
        return self.value
    def getWeight(self):
        return self.weight
    def __str__(self):
        result = '<' + self.name + ', ' + str(self.value)\
            + ', ' + str(self.weight) + '>'
        return result
    
def value(item):
    return item.getValue()
def weightInverse(item):
    return 1.0/item.getWeight()
def density(item):
    return item.getValue()/item.getWeight()

def greedy(items, maxWeight, keyFunction):
    """itemsはリスト、maxWeight>=0とし、
            KeyFunctionはitemsの要素を数にマップする"""
    itemsCopy = sorted(items, key=keyFunction, reverse = True) # sortedはsortと違い新たなリストを作る
    result = []
    totalValue, totalWeight = 0.0, 0.0
    for i in range(len(itemsCopy)):
        if (totalWeight + itemsCopy[i].getWeight()) <= maxWeight:
            result.append(itemsCopy[i])
            totalWeight += itemsCopy[i].getWeight()
            totalValue += itemsCopy[i].getValue()
    return (result, totalValue)

def buildItems():
    names = ['clock', 'painting', 'radio', 'vase', 'book', 'computer']
    values = [175, 90, 20, 50, 10, 200]
    weights = [10, 9, 4, 2, 1, 20]
    Items = []
    for i in range(len(values)):
        Items.append(Item(names[i], values[i], weights[i]))
    return Items

# 貪欲法
def testGreedy(items, maxWeight, keyFunction):
    taken, val = greedy(items, maxWeight, keyFunction)
    print('Total value of items taken is', val)
    for item in taken:
        print(' ', item)

def testGreedys(maxWeight = 20):
    items = buildItems()
    print('Use greedy by value to fill knapsack of size', maxWeight)
    testGreedy(items, maxWeight, value)
    print('\nUse greedy by weight to fill knapsack of size', maxWeight)
    testGreedy(items, maxWeight, weightInverse)
    print('\nUse greedy by density to fill knapsack of size', maxWeight)
    testGreedy(items, maxWeight, density)

testGreedys()

Use greedy by value to fill knapsack of size 20
Total value of items taken is 200.0
  <computer, 200, 20>

Use greedy by weight to fill knapsack of size 20
Total value of items taken is 170.0
  <book, 10, 1>
  <vase, 50, 2>
  <radio, 20, 4>
  <painting, 90, 9>

Use greedy by density to fill knapsack of size 20
Total value of items taken is 255.0
  <vase, 50, 2>
  <clock, 175, 10>
  <book, 10, 1>
  <radio, 20, 4>


In [2]:
# ナップサック問題の最適解

# 力づくのアプローチ
def getBinaryRep(n, numDigits):
    """nとnumDigitsを非負のint型とする
            nの値を, numDigits桁の２進数で表す文字列を返す"""
    result = ''
    while n > 0:
        result = str(n%2) + result
        n = n//2
    if len(result) > numDigits:
        raise ValueError('not enough digits')
    for i in range(numDigits - len(result)):
        result = '0' + result
    return result

def genPowerset(L):
    """Lをリストとする
            Lの要素のすべての可能な組合せからなるリストを返す
            例えばLが[1, 2]ならば
            [], [1], [2], [1,2]を要素に持つリストを返す"""
    powerset = []
    for i in range(0, 2**len(L)):
        binStr = getBinaryRep(i, len(L))
        subset = []
        for j in range(len(L)):
            if binStr[j] == '1':
                subset.append(L[j])
        powerset.append(subset)
    return powerset

# 計算量はO(n2^n)
def chooseBest(pset, maxWeight, getVal, getWeight):
    bestVal = 0.0
    bestSet = None
    for items in pset:
        itemsVal = 0.0
        itemsWeight = 0.0
        for item in items:
            itemsVal += getVal(item)
            itemsWeight += getWeight(item)
        if itemsWeight <= maxWeight and itemsVal > bestVal:
            bestVal = itemsVal
            bestSet = items
    return (bestSet, bestVal)

def testBest(maxWeight = 20):
    items = buildItems()
    pset = genPowerset(items)
    taken, val = chooseBest(pset, maxWeight, Item.getValue, Item.getWeight)
    print('Total value of items taken is', val)
    for item in taken:
        print(item)
        
testBest()    

Total value of items taken is 275.0
<clock, 175, 10>
<painting, 90, 9>
<book, 10, 1>


In [3]:
#ナップサック問題を深さ優先で解く
def maxVal(toConsider, avail):
    """toConsiderを品物リスト,availを重さとする
        それらをパラメータとするナップサック問題の解である
        総価値と品物のリストからなるタプルを返す"""
    if toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getWeight() > avail:
        #右側の分岐のみを探索する
        result =maxVal(toConsider[1:], avail)
    else:
        nextItem = toConsider[0]
        #左側の分岐を探索する
        withVal, withToTake = maxVal(toConsider[1:], avail - nextItem.getWeight())
        withVal += nextItem.getValue()
        #右側の分岐を探索する
        withoutVal, withoutToTake = maxVal(toConsider[1:], avail)
        #よりよい分岐を選択
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result

In [4]:
#動的計画法を用いた場合
# 疑似多項式と呼ばれるクラスに属する
# 大雑把には,fastMaxValはavailの撮りうる値を表現するのに必要なビット数の指数オーダーの手法である
def fastMaxVal(toConsider, avail, memo = {}):
    """toConsiderを品物のリスト,availを重さ,memoは再帰呼出しによってのみ使われるとする
        それらをパラメータとするナップサック問題の解である
        総価値と品物のリストからなるタプルを返す"""
    if (len(toConsider), avail) in memo:
        result = memo[(len(toConsider), avail)]
    elif toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getWeight() > avail:
        #右側の分岐のみを探索する
        result = fastMaxVal(toConsider[1:], avail, memo)
    else:
        nextItem = toConsider[0]
        #左側の分岐を探索する
        withVal, withToTake = fastMaxVal(toConsider[1:], \
                                        avail - nextItem.getWeight(), memo)
        withVal += nextItem.getValue()
        #右側の分岐を探索する
        withoutVal, withoutToTake = fastMaxVal(toConsider[1:], avail, memo)
        
        #よりよい分岐を選択
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    memo[(len(toConsider), avail)] = result
    return result

In [5]:
import random
def smallTest():
    names = ['a', 'b', 'c', 'd']
    vals = [6, 7, 8, 9]
    weights = [3, 3, 2, 5]
    Items = []
    for i in range(len(vals)):
        Items.append(Item(names[i], vals[i], weights[i]))
    val, taken = maxVal(Items, 5)
    for item in taken:
        print(item)
    print('Total value of items taken =', val)

def buildManyItems(numItems, maxVal, maxWeight):
    items = []
    for i in range(numItems):
        items.append(Item(str(i),
                                     random.randint(1, maxVal),
                                     random.randint(1, maxWeight)))
    return items

def bigTest(numItems):
    items = buildManyItems(numItems, 10, 10)
    val, taken = fastMaxVal(items, 40)
    print('Items Taken')
    for item in taken:
        print(item)
    print('Total value of items taken =', val)

smallTest()
bigTest(10)
bigTest(40)

<c, 8, 2>
<b, 7, 3>
Total value of items taken = 15
Items Taken
<9, 9, 10>
<7, 6, 2>
<6, 8, 4>
<4, 7, 9>
<3, 7, 6>
<2, 4, 6>
<1, 1, 1>
Total value of items taken = 42
Items Taken
<6, 8, 4>
<34, 4, 2>
<30, 6, 5>
<29, 7, 4>
<26, 10, 3>
<23, 6, 1>
<22, 9, 2>
<20, 7, 2>
<19, 10, 1>
<11, 9, 3>
<8, 10, 6>
<7, 10, 6>
<3, 6, 1>
Total value of items taken = 102


In [6]:
getBinaryRep(255, 8)
ret = genPowerset([1,2,3,4,5])
print(ret)
print('Combination pattern is', len(ret))
# 組合せの場合の数: 1 + 5C5 + 5C4 + 5C3 + 5C2 + 5C1
# = 1 + 1 + 5 + 10 + 10 + 5
# = 32

[[], [5], [4], [4, 5], [3], [3, 5], [3, 4], [3, 4, 5], [2], [2, 5], [2, 4], [2, 4, 5], [2, 3], [2, 3, 5], [2, 3, 4], [2, 3, 4, 5], [1], [1, 5], [1, 4], [1, 4, 5], [1, 3], [1, 3, 5], [1, 3, 4], [1, 3, 4, 5], [1, 2], [1, 2, 5], [1, 2, 4], [1, 2, 4, 5], [1, 2, 3], [1, 2, 3, 5], [1, 2, 3, 4], [1, 2, 3, 4, 5]]
Combination pattern is 32
